In [13]:
import pandas as pd

In [14]:
# Daten laden
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [2]:
# CSVs laden
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")
projekte_df = pd.read_csv("../data/projekte.csv")

In [15]:
# Kompetenzspalte vorbereiten
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(",\s*")

In [16]:
# Belegungstracker: person_id → monat (z. B. "03/2025") → belegte Kapazität
belegungen = {}

In [18]:
# Ergebnisse
matching_ergebnisse = []

for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    ta_monatsaufwand = ta["aufwand"] / ta_monate
    restaufwand = ta["aufwand"]

    kandidaten = []

    # Filtere Personen nach Kompetenz
    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        person_id = person["id"]
        verfügbar = True
        verfügbarkeit_summe = 0

        for monat in range(ta_start, ta_ende + 1):
            monat_label = f"{monat:02d}/2025"
            verfügbarkeit = person.get(f"verfuegbarkeit_{monat_label}", 0)
            belegt = belegungen.get(person_id, {}).get(monat_label, 0)
            rest = verfügbarkeit - belegt
            if rest <= 0:
                verfügbar = False
                break
            verfügbarkeit_summe += rest

        if verfügbar and verfügbarkeit_summe > 0:
            score = round(verfügbarkeit_summe * person["zeitbudget"], 2)
            kandidaten.append((person, score, verfügbarkeit_summe))

    # Sortiere nach bester nutzbarer Kapazität
    kandidaten = sorted(kandidaten, key=lambda x: x[1], reverse=True)

    for person, score, verf_summe in kandidaten:
        if restaufwand <= 0:
            break

        pid = person["id"]
        if pid not in belegungen:
            belegungen[pid] = {}

        verfügbarkeit_gesamt = 0
        eintrag_monate = {}
        for monat in range(ta_start, ta_ende + 1):
            monat_label = f"{monat:02d}/2025"
            verfügbar = person[f"verfuegbarkeit_{monat_label}"]
            belegt = belegungen[pid].get(monat_label, 0)
            rest = max(0, verfügbar - belegt)
            eintrag_monate[monat_label] = rest
            verfügbarkeit_gesamt += rest

        if verfügbarkeit_gesamt == 0:
            continue

        anteil = min(restaufwand, verfügbarkeit_gesamt)
        monatsanteil = anteil / ta_monate

        # Zuweisung speichern
        matching_ergebnisse.append({
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "projekt_id": ta["projekt_id"],
            "kompetenz": ta["kompetenz"],
            "person_id": pid,
            "name": person["name"],
            "zugewiesener_aufwand": round(anteil, 2)
        })

        # Tracker aktualisieren
        for monat_label in eintrag_monate:
            verbrauch = min(monatsanteil, eintrag_monate[monat_label])
            belegungen[pid][monat_label] = belegungen[pid].get(monat_label, 0) + verbrauch

        restaufwand -= anteil

In [ ]:
print(f"Verfügbare Kandidaten für: {ta['bezeichnung']} → {len(kandidaten)}")

Verfügbare Kandidaten für: Thermodynamische Bewertung → 17


In [20]:
# Ergebnis speichern
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print("Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")

Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'
